In [4]:
import requests
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def fetch_ipo_data_robust(total_pages_to_scrape=83): # 目标设为83页
    url = "https://datacenter-web.eastmoney.com/api/data/v1/get"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': 'https://data.eastmoney.com/',
        'Accept': 'application/json, text/javascript, */*; q=0.01'
    }

    # ---------------------------------------------------------
    # 核心升级：配置自动重试机制
    # ---------------------------------------------------------
    session = requests.Session()
    # max_retries=3: 失败后重试3次
    # backoff_factor=1: 每次重试间隔时间递增 (1s, 2s, 4s...)
    retry_strategy = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)

    all_data_list = []
    
    # 根据你截图的正确参数
    params_base = {
        'sortColumns': 'UPDATE_DATE,ORG_CODE',
        'sortTypes': '-1,-1',
        'pageSize': '50',
        'reportName': 'RPT_IPO_INFOALLNEW',
        'columns': 'ALL',
        'source': 'WEB',
        'client': 'WEB'
    }

    for page in range(1, total_pages_to_scrape + 1):
        print(f"[{page}/{total_pages_to_scrape}] 正在爬取...", end="")
        
        params = params_base.copy()
        params['pageNumber'] = page

        try:
            # 注意：这里把 timeout 增加到了 20 秒，给服务器更多反应时间
            # 使用 session.get 而不是 requests.get
            response = session.get(url, headers=headers, params=params, timeout=20)
            
            if response.status_code == 200:
                data_json = response.json()
                
                if data_json.get('result') and data_json['result'].get('data'):
                    items = data_json['result']['data']
                    print(f" 成功获取 {len(items)} 条")
                    
                    for item in items:
                        row = {
                            '企业名称': item.get('DECLARE_ORG') or item.get('SECURITY_NAME'), 
                            '注册地': item.get('REG_ADDRESS'),
                            '行业': item.get('CSRC_INDUSTRY'),
                            '保荐机构': item.get('RECOMMEND_ORG'),
                            '律师事务所': item.get('LAW_FIRM'),
                            '会计师事务所': item.get('ACCOUNT_FIRM'),
                            '拟上市板': item.get('PREDICT_LISTING_MARKET'),
                            '更新日期': item.get('UPDATE_DATE'),
                            '受理日期': item.get('ACCEPT_DATE'),
                            '当前状态': item.get('STATE')
                        }
                        all_data_list.append(row)
                else:
                    print("\n数据已爬完（接口未返回数据）。")
                    break
            else:
                print(f" 接口报错: {response.status_code}")
                # 如果遇到500等错误，最好不要强行继续，但这里我们记录后继续尝试下一页
                
            # 正常翻页等待1秒
            time.sleep(1)

        except Exception as e:
            print(f"\n第 {page} 页彻底失败 (已重试3次): {e}")
            # 即使这一页失败了，也继续爬下一页，不要 break
            continue

    # 保存数据
    if all_data_list:
        df = pd.DataFrame(all_data_list)
        print(f"\n>>> 爬取结束，共获取 {len(df)} 条数据 <<<")
        
        filename = "ipo_full_data.xlsx"
        df.to_excel(filename, index=False)
        print(f"文件已保存为: {filename}")
    else:
        print("未抓取到任何数据。")

if __name__ == "__main__":
    fetch_ipo_data_robust()

[1/83] 正在爬取... 成功获取 50 条
[2/83] 正在爬取... 成功获取 50 条
[3/83] 正在爬取... 成功获取 50 条
[4/83] 正在爬取... 成功获取 50 条
[5/83] 正在爬取... 成功获取 50 条
[6/83] 正在爬取... 成功获取 50 条
[7/83] 正在爬取... 成功获取 50 条
[8/83] 正在爬取... 成功获取 50 条
[9/83] 正在爬取... 成功获取 50 条
[10/83] 正在爬取... 成功获取 50 条
[11/83] 正在爬取... 成功获取 50 条
[12/83] 正在爬取... 成功获取 50 条
[13/83] 正在爬取... 成功获取 50 条
[14/83] 正在爬取... 成功获取 50 条
[15/83] 正在爬取... 成功获取 50 条
[16/83] 正在爬取... 成功获取 50 条
[17/83] 正在爬取... 成功获取 50 条
[18/83] 正在爬取... 成功获取 50 条
[19/83] 正在爬取... 成功获取 50 条
[20/83] 正在爬取... 成功获取 50 条
[21/83] 正在爬取... 成功获取 50 条
[22/83] 正在爬取... 成功获取 50 条
[23/83] 正在爬取... 成功获取 50 条
[24/83] 正在爬取... 成功获取 50 条
[25/83] 正在爬取... 成功获取 50 条
[26/83] 正在爬取... 成功获取 50 条
[27/83] 正在爬取... 成功获取 50 条
[28/83] 正在爬取... 成功获取 50 条
[29/83] 正在爬取... 成功获取 50 条
[30/83] 正在爬取... 成功获取 50 条
[31/83] 正在爬取... 成功获取 50 条
[32/83] 正在爬取... 成功获取 50 条
[33/83] 正在爬取... 成功获取 50 条
[34/83] 正在爬取... 成功获取 50 条
[35/83] 正在爬取... 成功获取 50 条
[36/83] 正在爬取... 成功获取 50 条
[37/83] 正在爬取... 成功获取 50 条
[38/83] 正在爬取... 成功获取 50 条
[39/83] 正在爬取... 成功获取 